In [ ]:
# Imports
%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib import pyplot as plt
import csv
import importlib
from tqdm import tqdm
from scripts import proj1_helpers, helpers
from scripts import implementation, feature_processing, k_fold, model_linear, model_logistic
import numpy as np

In [ ]:
train_path = '../data/train.csv'
test_path  = '../data/test.csv'
output_path = '../data/linear_processed_submission.csv'

In [ ]:
# loading data
y, X, idx = proj1_helpers.load_csv_data(train_path)
y_t, X_t, ids_t = proj1_helpers.load_csv_data(test_path)

In [251]:
X_p, (x_mean, x_std) = feature_processing.process_X(X)
X_t_p, _ = feature_processing.process_X(X_t, (x_mean, x_std))

100%|██████████| 5/5 [00:18<00:00,  3.61s/it]


In [ ]:
#Linear
model = implementation.ridge_regression
loss = model_linear.compute_accuracy_loss

idx_min_l, rmse_all_l, lambdas_l = k_fold.cross_validation_select(X_p, y, model, loss, seed = 1,
                                                            k_fold = 3, N = 20, do_plot = True, do_tqdm = True)

In [ ]:
lambda_ = lambdas[idx_min]
print(lambda_)
w, l = implementation.ridge_regression(y, X_p, lambda_ = 0)

  5%|▌         | 5/100 [00:01<00:26,  3.64epoch/s, acc=-0.79, diff=2.33, grad=173, loss=8.9e+04, w=171]/home/sergei/Documents/repos/git/EPFL/ML/Project1/src/scripts/model_logistic.py:12: RuntimeWarning: overflow encountered in exp
  return -np.sum(np.multiply((tx @ w).flatten(), y.flatten())) + np.sum(np.log1p(np.exp(tx @ w)))
 16%|█▌        | 16/100 [00:04<00:20,  4.12epoch/s, acc=-0.81, diff=0.0351, grad=13.9, loss=inf, w=56.5] 


In [314]:
#Logistic
y_01 = helpers.y_to_01(np.array([y]).T)
np.random.seed(1)
w0 = np.zeros((X_p.shape[1], 1))#np.random.randn(X_p.shape[1], 1)
model = implementation.reg_logistic_regression_newton_batch
model_args = {'initial_w': w0, 'max_iters': 100, 'gamma': .5, 'debug': False, 'batch_size': 50000}
#loss = model_logistic.reg_loss
loss = model_linear.compute_accuracy_loss

lambdas = np.logspace(-6, -3, 20)

idx_min, rmse_all, lambdas = k_fold.cross_validation_select(X_p, y_01, model, loss, kw_model = model_args, seed = 1,
                                                            k_fold = 3, lambdas = lambdas, do_plot = True, do_tqdm = False)

In [316]:
lambda_ = lambdas[idx_min]
print(lambda_)

0.000233572146909


In [ ]:
#w1, l = implementation.reg_logistic_regression(y_01, X_p, lambda_ = 0.1,
#                                                     initial_w = w0, max_iters = 100, gamma = 1e-6, debug = False)

In [198]:
w1, l = implementation.reg_logistic_regression_batch(y_01, X_p, lambda_ = 0.1,
                                                     initial_w = w0, batch_size = 50000,
                                                     max_iters = 300, gamma = 1e-5, debug = False)

 83%|████████▎ | 250/300 [00:39<00:07,  6.67epoch/s, acc=-0.78, grad=869, loss=1.15e+05, w=2.54]     


In [336]:
np.random.seed(42)
w2, l = implementation.reg_logistic_regression_newton_batch(y_01, X_p, lambda_ = 0.000233572146909,
                                                     initial_w = w1, batch_size = 50000,
                                                     max_iters = 100, gamma = .1, debug = False)

  0%|          | 0/100 [00:00<?, ?epoch/s]/home/sergei/Documents/repos/git/EPFL/ML/Project1/src/scripts/model_logistic.py:12: RuntimeWarning: overflow encountered in exp
  return -np.sum(np.multiply((tx @ w).flatten(), y.flatten())) + np.sum(np.log1p(np.exp(tx @ w)))
 84%|████████▍ | 84/100 [00:33<00:05,  3.00epoch/s, acc=-0.81, diff=3.42e-05, grad=7.35, loss=inf, w=122]


In [337]:
y_pred = proj1_helpers.predict_labels(w, X_t_p)

In [338]:
proj1_helpers.create_csv_submission(ids_t, y_pred, output_path)

In [ ]:
tqdm.get_lock().locks = []

In [253]:
np.mean(y_pred>0)

0.30083521341409764